In [2]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

import pandas as pd

from joblib import Parallel, delayed

from scipy.interpolate import RectBivariateSpline
from skimage.transform import rescale, resize

In [3]:
stack = 'MD589'
# stack = 'MD594'
dm = DataManager(stack=stack)

label_polygons = pd.read_hdf('/home/yuncong/Brain/database/'+stack+'_label_polygons.h5', 'label_polygons')

In [4]:
label_polygons

,10N,12N,3N,4N,5N,5TT,6N,7N,7n,9N,...,VOri,VOrp,VTg,Vcd,dtgx,mlf,pyx,scp,sp5,vtgx
110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"[[8477, 8197], [8354, 8311], [8235, 8491], [82...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"[[8439, 8194], [8328, 8246], [8188, 8396], [81...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"[[8414, 8181], [8244, 8277], [8117, 8378], [80...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"[[7813, 8961], [7792, 8795], [7841, 8628], [79...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# labels =  ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
                     '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']


other_labels = [l for l in label_polygons.dropna().keys() if l not in labels]

# label_dict = dict([(l,i) for i, l in enumerate(labels)])

label_dict = dict([(l,i) for i, l in enumerate(labels)] + \
                  zip(other_labels, range(len(labels), len(labels)+len(other_labels))))

colors = (np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')*255).astype(np.uint8)
colors[label_dict['BackG']] = 255

In [6]:
labelMapLossless_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_labelMapsLossless/%(stack)s/' % {'stack': stack}
if not os.path.exists(labelMapLossless_dir):
    os.makedirs(labelMapLossless_dir)

In [7]:
from matplotlib.path import Path
from itertools import product

In [8]:
# only 9 (19) classes
for sec in label_polygons.index:
    print sec
    
    available_labels = label_polygons.loc[sec].dropna().index
    
    valid_labels = set(available_labels) & set(labels)
    
    if len(valid_labels) == 0:
        continue
    
    label_map = np.zeros((dm.image_height, dm.image_width), np.int8)
    for l in valid_labels:
        print l
        vs = label_polygons[l][sec]
        xmin, ymin = np.min(vs, axis=0)
        xmax, ymax = np.max(vs, axis=0)
                
        if xmin > dm.image_width - 1 or ymin > dm.image_height - 1:
            print 'min coordinates exceed image boundary'
            continue
        
        xmax = min(xmax, dm.image_width-1)
        ymax = min(ymax, dm.image_height-1)
        
#         import time
#         t = time.time()
        
#         points_to_test = np.array(list(product(range(ymin, ymax+1), range(xmin, xmax+1))))[:,::-1]
        xs, ys = np.meshgrid(range(xmin, xmax+1), range(ymin, ymax+1))
        points_to_test = np.c_[xs.flat, ys.flat]
        
#         print time.time() - t
        
        is_contained = Path(vs).contains_points(points_to_test)
        label_map[points_to_test[is_contained, 1], points_to_test[is_contained, 0]] = label_dict[l]
    
    
    bp.pack_ndarray_file(label_map, os.path.join(labelMapLossless_dir, 
                                     '%(stack)s_%(sec)04d_labelMapLossless.bp' % \
                                    {'stack': stack, 'sec': sec}))

#     np.savez_compressed(os.path.join(labelMapLossless_dir, 
#                                      '%(stack)s_%(sec)04d_labelMapLossless' % \
#                                     {'stack': stack, 'sec': sec}), 
#                         label_map)

110
111
113
115
117
119
121
123
125
127
129
131
sp5
133
sp5
135
sp5
137
sp5
139
sp5
141
sp5


KeyboardInterrupt: 

In [ ]:
# all landmarks
for sec in label_polygons.index:
    
    print sec
    
    available_labels = label_polygons.loc[sec].dropna().index
    
    valid_labels = set(available_labels)
    
    if len(valid_labels) == 0:
        continue
    
    label_map = np.zeros((dm.image_height, dm.image_width), np.int8)
    for l in valid_labels:
        print l
        vs = label_polygons[l][sec]
        xmin, ymin = np.min(vs, axis=0)
        xmax, ymax = np.max(vs, axis=0)
        
        if xmin > dm.image_width - 1 or ymin > dm.image_height - 1:
            print 'min coordinates exceed image boundary'
            continue
        
        xmax = min(xmax, dm.image_width-1)
        ymax = min(ymax, dm.image_height-1)
        
        points_to_test = np.array(list(product(range(ymin, ymax+1), range(xmin, xmax+1))))[:,::-1]
        is_contained = Path(vs).contains_points(points_to_test)
        
        label_map[points_to_test[is_contained, 1], 
                  points_to_test[is_contained, 0]] = label_dict[l]
    
    np.savez_compressed(os.path.join(labelMapLossless_dir, 
                                     '%(stack)s_%(sec)04d_labelMapLossless_AllClasses' % \
                                    {'stack': stack, 'sec': sec}), 
                        label_map)